In [105]:
import os
import openai

from dotenv import load_dotenv
load_dotenv('myenv/.env')
a=os.environ.get('OPENAI_API_KEY')
openai.api_key=a


In [89]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["/Users/eshantdas/Desktop/Deeplearning_hehehe/Advance_lag/ebook.pdf"]
).load_data()

In [106]:

#Merge all of this into a single large document

from llama_index.core import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

## `Automerge Retriever`

In [107]:
from llama_index.core.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [108]:
nodes = node_parser.get_nodes_from_documents([document])

In [109]:
#get the leaf nodes

from llama_index.core.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(type(leaf_nodes))
print(leaf_nodes[2])
print(type(leaf_nodes[2]))
print('------')
print('------')
print('------')
print('------')
print(leaf_nodes[2].text)

<class 'list'>
Node ID: 60483177-fe18-466c-b57d-74fb670bc1f8
Text: Chapter 11: Overcoming Imposter Syndrome. Final Thoughts: Make
Every Day Count.LEARNING PROJECTS JOB  PAGE 4Coding AI Is the New
Literacy Today we take it for granted that many people know how to
read and write. Someday, I hope,  it will be just as common that
people know how to write code, specifically for AI. Several hundred
years ago, society...
<class 'llama_index.core.schema.TextNode'>
------
------
------
------
Chapter 11: Overcoming Imposter Syndrome.
Final Thoughts: Make Every Day Count.LEARNING
PROJECTS
JOB

PAGE 4Coding AI Is the New Literacy
Today we take it for granted that many people know how to read and write. Someday, I hope, 
it will be just as common that people know how to write code, specifically for AI.
Several hundred years ago, society didn’t view language literacy as a necessary skill. A small 
number of people learned to read and write, and everyone else let them do the reading and 
writing.


In [94]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[2].parent_node.node_id]
print(parent_node)
print(parent_node.text)

Node ID: bdf7a490-eb11-4fc9-ae3f-09a1b6d1917a
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide   PAGE 2"AI is the new
electricity. It will  transform and improve  all areas of human life."
Andrew Ng  PAGE 3Table of  ContentsIntroduction: Coding AI is the New
Literacy. Chapter 1: Three Steps to Career Growth. Chapter 2: Learning
T...
PAGE 1Founder, DeepLearning.AICollected Insights
from Andrew Ng
How to 
Build
Your
Career
in AIA Simple Guide


PAGE 2"AI is the new 
electricity. It will 
transform and improve 
all areas of human life."
Andrew Ng

PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Career Growth.
Chapter 2: Learning Technical Skills for a 
Promising AI Career.
Chapter 3: Should You Learn Math to Get a Job 
in AI?
Chapter 4: Scoping Successful AI Projects.
Chapter 5: Finding Projects that Complement 
Your Career Goals.
Chapter 6: Building a Portfolio of Projects th

## `BUILDING THE INDEX`

In [95]:
from utils import build_sentence_window_index
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model=OpenAIEmbedding(api_key=a)

# embed_model = AzureOpenAIEmbedding(
#     model="text-embedding-ada-002",
#     deployment_name="ada-embedding-model",
#     api_key='af2c3176c0d04bbc9f3db40e570869e1',
#     azure_endpoint='https://iprodata.openai.azure.com',
#     api_version='2023-05-15'
# )
# llm = AzureOpenAI(
#     model="gpt-4-32k",
#     deployment_name="gpt432",
#     api_key='72b9712ff8ec450a9d6000898d9cba94',
#     azure_endpoint='https://uksouth-gpt432.openai.azure.com',
#     api_version='2023-05-15')

In [96]:
from llama_index.core import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

/var/folders/tv/gkhqvlr92js55bf3s7q8jr000000gp/T/ipykernel_47158/2158502765.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  auto_merging_context = ServiceContext.from_defaults(


In [97]:
from llama_index.core import VectorStoreIndex, StorageContext

#Here we construct our index
#only the leaf nodes are indexed and all the other nodes are stored in a doc store and are retrieved dynamically during retrieval

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index/random_string2")

# `Optional`

In [104]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index/random_string/")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index/random_string/"),
        service_context=auto_merging_context
    )


# `---------------------------------------------------------------------------------------------------`

In [110]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [111]:
auto_merging_response = auto_merging_engine.query(
    "What is the importance of networking in AI?"
)

In [112]:
from llama_index.core.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking in AI is crucial as it helps individuals build a strong professional community that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive guidance, advice, and potential referrals to employers. Additionally, networking allows individuals to showcase their expertise, receive encouragement for further development, and feel a sense of belonging within the AI community. This support system can be instrumental in advancing one's career and navigating the challenges and opportunities within the AI industry.

In [114]:
import os

from llama_index.core import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model,
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [116]:


index = build_automerging_index(
    [document],
    llm=llm,
    embed_model=embed_model,
    save_dir="./merging_index/random_string2/"
)


/var/folders/tv/gkhqvlr92js55bf3s7q8jr000000gp/T/ipykernel_47158/983829541.py:28: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  merging_context = ServiceContext.from_defaults(


In [58]:
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

In [59]:
auto_merging_response = query_engine.query(
    "What is the importance of networking in AI?"
)

In [60]:
from llama_index.core.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Networking in AI is crucial as it allows individuals to build a strong professional network that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive advice, referrals to potential employers, and stay updated on the latest trends and technologies in AI. Additionally, networking can help individuals establish themselves as experts in the field, leading to recognition and further opportunities for growth and development.